# Transportation CO2 Emission  

In [ ]:
!pip install geopy

In [ ]:
# Import libraries.
# import os
# import glob
import numpy as np
import pandas as pd
import seaborn as sns
import networkx as nx
import matplotlib.pyplot as plt
from geopy.distance import geodesic
from geopy.geocoders import Nominatim

## Import data

In [ ]:
# Load train data.
ROOTPATH_DATA = r"../datasets"
train_df = pd.read_csv(ROOTPATH_DATA+"/train.csv", sep=",")

In [ ]:
# Processing.
columns_to_clean = ['Month 2', 'Month 3', 'Month 4']
train_df[columns_to_clean] = (
    train_df[columns_to_clean]
    .apply(pd.to_numeric, errors='coerce')
    .astype('Int64')
)

train_df['Country'] = train_df['Country'].str.upper()
train_df['Site'] = train_df['Site'].str.upper()

In [ ]:
# Load country_code data.
country_code_df = pd.read_csv(
    "../datasets/countries_codes_and_coordinates.csv",
    usecols=["Country",
             "Alpha-2 code",
             "Latitude (average)",
             "Longitude (average)"]
)

# Clean and process data.
country_code_df["Alpha-2 code"] = (
    country_code_df["Alpha-2 code"]
    .str.replace('"', '')
    .str.strip()
    .astype(str)
)

# Use pd.eval to safely evaluate expressions.
country_code_df['Coordinates'] = list(zip(
    country_code_df['Latitude (average)'].str.strip().apply(lambda x: eval(x)).astype(float),
    country_code_df['Longitude (average)'].str.strip().apply(lambda x: eval(x)).astype(float)
))

# Drop unnecessary columns and rename.
country_code_df.drop(
    columns=["Longitude (average)", "Latitude (average)"], inplace=True
)
country_code_df.rename(columns={"Alpha-2 code": "Code", "Country": "Name"}, 
                       inplace=True)

country_code_df.head()


In [ ]:
def extract_country_code(site):
    return 'US' if site.startswith('OOS') else site.split('_')[0]


# Add a new column 'Site Country' to train_df
train_df['Site Country'] = train_df['Site'].apply(extract_country_code)

# Apply str.upper() to entire columns
train_df['Country'] = train_df['Country'].str.upper()
train_df['Site Country'] = train_df['Site Country'].str.upper()

# Merge train_df with country_code_df to get country names
merged_df = pd.merge(train_df, country_code_df, left_on='Site Country', right_on='Code', how='left')

# Drop unnecessary columns and rename columns
merged_df.drop(['Site Country', 'Code'], axis=1, inplace=True)
merged_df.rename(columns={'Name': 'Site Country', 'Coordinates': 'Coordinates Site'}, inplace=True)

# Merge train_df with country_code_df to get country names
merged_df = pd.merge(merged_df, country_code_df, left_on='Country', right_on='Code', how='left')

# Drop unnecessary columns and rename columns
merged_df.drop(['Country', 'Code'], axis=1, inplace=True)
merged_df.rename(columns={'Name': 'Country', 'Coordinates': 'Coordinates Country'}, inplace=True)

# Display the final DataFrame
merged_df.head(5)


In [ ]:
train_df = merged_df.copy()

# Use a single replace call with a dictionary
country_name_mapping = {"Viet Nam": "Vietnam", "Taiwan, Province of China": "Taiwan"}
train_df.replace({"Country": country_name_mapping, "Site Country": country_name_mapping}, inplace=True)

# Print sorted unique values in one step
print(sorted(train_df["Country"].unique()))
print(sorted(train_df["Site Country"].unique()))

In [ ]:
train_df.head(2)

## Compute distance and estimtates CO2 emissions

## Plot Transportation Lines Graph

In [ ]:
# Aggregate sales data by product, site and country
aggregated_df = train_df.groupby(['id_product', 'Site Country', 'Country']).agg({
    'Month 1': 'sum',
    'Month 2': 'sum',
    'Month 3': 'sum'
}).reset_index()

aggregated_df['Sales'] = aggregated_df[['Month 1', 'Month 2', 'Month 3']].sum(axis=1)

# Drop the individual month columns
aggregated_df = aggregated_df.drop(['Month 1', 'Month 2', 'Month 3'], axis=1)

# DON'T COMMENT BELOW FILTER if you want to remove products that have Site Country equal to Country sold
# aggregated_df = aggregated_df[aggregated_df['Site Country'] != aggregated_df['Country']]

# DON'T COMMENT BELOW FILTER if you want to remove products that have Site Country equal to Country sold
# aggregated_df = aggregated_df[aggregated_df['Site Country'] == aggregated_df['Country']]

aggregated_df.head(5)

In [ ]:
cols = [
    "Site Country",
    "Country",
    "Coordinates Site",
    "Coordinates Country"
]
country_coordinates_df = train_df[cols]
country_coordinates_df = country_coordinates_df.drop_duplicates()
print(country_coordinates_df.shape)
country_coordinates_df.head()

In [ ]:
aggregated_df = aggregated_df.rename(columns={'Country':'CountryLeft', 'Site Country':'Site CountryLeft'})
aggregated_df.dtypes

In [ ]:
# Merge the coordinates with the main dataframe based on the 'Country' column
aggregated_df = pd.merge(aggregated_df, country_coordinates_df, left_on=['Site CountryLeft', 'CountryLeft'], 
                         right_on=['Site Country', 'Country'], how='left')

In [ ]:
aggregated_df.head()

In [ ]:
aggregated_df[['Coordinates Country', 'Coordinates Site']].isna().sum()

In [ ]:
# nan_rows = aggregated_df[aggregated_df['Country'].isna()]
nan_rows = aggregated_df[aggregated_df['Coordinates Country'] == 'nan']
nan_rows2 = aggregated_df[aggregated_df['Coordinates Site'] == 'nan']

print(nan_rows, nan_rows2)

In [ ]:
aggregated_df.drop(columns=["Site CountryLeft", "CountryLeft"], inplace=True)

In [ ]:
# Function to calculate distance using Haversine formula
def calculate_distance(row):
    return geodesic((row['Coordinates Country']), (row['Coordinates Site'])).kilometers

# Apply the function to create a new 'distance' column
aggregated_df['distance'] = aggregated_df.apply(calculate_distance, axis=1)
print(aggregated_df.shape)
aggregated_df.head()

In [ ]:
aggregated_df[aggregated_df["distance"] > 0]["distance"].min()

In [ ]:
pop_df = pd.read_csv(ROOTPATH_DATA+"/extra-dataset/population-data.csv", usecols=["country", "landAreaKm", "cca2"])
pop_df.head()

In [ ]:
# Merge on 'Site Country' and 'country', perform a left join
aggregated_df = pd.merge(aggregated_df, pop_df, left_on='Site Country', right_on='country', how='left')
aggregated_df = aggregated_df.drop(['country', 'cca2'], axis=1)
aggregated_df.head()

[ourworldindata](https://ourworldindata.org/grapher/carbon-footprint-travel-mode)
[ourworldindata](https://ourworldindata.org/grapher/carbon-footprint-travel-mode)

|Transportation Mode| Consumption Emission|
|------|-------------|
|Plane | 175 gCO2e/km|
|Motorbike | 92 gCO2e/km|
|Petrol car | 90 gCO2e/km|
|Diesel car| 90 gCO2e/km|
|Train| 28 gCO2e/km|
|Electric car| 29 gCO2e/km|
|Coach| 22 gCO2e/km|
|Long Haul Truck| 57 gCO2e/km|
|Roll-on/Roll-off Ferry| 52 gCO2e/km|

In [ ]:
aggregated_df["distance"] = np.where(
    aggregated_df["distance"] == 0,  # condition
    np.sqrt(aggregated_df["landAreaKm"]),  # if true
    aggregated_df["distance"]  # otherwise
)

In [ ]:
def calculate_co2_emission_ratio(row, vehicle_co2_emission, french_co2_emission):
    if row['Sales'] != 0:
        return (row['distance'] * vehicle_co2_emission) / (row['Sales'] * french_co2_emission)
    else:
        return (row['distance'] * vehicle_co2_emission) / french_co2_emission


vehicle_co2_emission = 80.4
french_co2_emission = 4.46
# Assuming you have a DataFrame named aggregated_df
aggregated_df['CO2 Emission'] = aggregated_df["distance"] * vehicle_co2_emission
aggregated_df['CO2 Emission Ratio French'] = aggregated_df.apply(
    lambda row: calculate_co2_emission_ratio(row, vehicle_co2_emission, french_co2_emission),
    axis=1
)

In [ ]:
aggregated_df.head(5)

In [ ]:
# Scatter plot of Sales vs. CO2 Emission
plt.scatter(aggregated_df['Sales'], aggregated_df['CO2 Emission'])
plt.title('Sales vs. CO2 Emission')
plt.xlabel('Sales')
plt.ylabel('CO2 Emissions per Person in France')
plt.savefig('mix sales_vs_co2_emission_plot.png')
plt.show()


In [ ]:
# Bar chart of Sales by Country
sales_by_country = aggregated_df.groupby('Country')['Sales'].sum().sort_values()
sales_by_country.plot(kind='bar', color='skyblue')
plt.title('Sales by Country')
plt.xlabel('Country')
plt.ylabel('Total Sales')
plt.savefig('mix sales_by_country_plot.png')
plt.show()

In [ ]:
# Bar chart of Sales by Country
stocks_by_country = aggregated_df.groupby('Site Country')['Sales'].sum().sort_values()
stocks_by_country.plot(kind='bar', color='skyblue')
plt.title('Total Stocks per Country')
plt.xlabel('Country')
plt.ylabel('Total Stocks in Sites')
plt.savefig('mix stocks_by_country_plot.png')
plt.show()

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import StandardScaler
import pandas as pd

# Assuming your DataFrame is named aggregated_df
# Group by product_id, summing up sales and taking the average of CO2 Emission Ratio French
aggregated_product_df = aggregated_df.groupby('id_product').agg({'Sales': 'sum', 'CO2 Emission Ratio French': 'mean'}).reset_index()

# Filter products with sales above 50
filtered_df = aggregated_product_df[aggregated_product_df['Sales'] > 50].copy()  # Create a copy to avoid SettingWithCopyWarning

# Standardize the 'Sales' and 'CO2 Emission Ratio French' columns
scaler = StandardScaler()
filtered_df.loc[:, ['Sales', 'CO2 Emission Over Median French Person Emission']] = scaler.fit_transform(filtered_df[['Sales', 'CO2 Emission Ratio French']])

# Plotting
plt.figure(figsize=(10, 8))
sns.scatterplot(x='Sales', y='CO2 Emission Ratio French', size='Sales', data=filtered_df, sizes=(20, 200), legend=True)

# Adding labels and title
plt.title('Standardized CO2 Emission vs. Sales (Aggregated by Product, Sales > 50)')
plt.xlabel('Standardized Sales')
plt.ylabel('Standardized CO2 Emission Ratio French')

# Display the plot
plt.show()


In [ ]:
# Plotting
plt.figure(figsize=(12, 8))
sns.barplot(x='Sales', y='Country', data=sales_by_country_df, palette='viridis', ci=None)

# Adding color scale legend
sm = plt.cm.ScalarMappable(cmap='viridis', norm=plt.Normalize(vmin=min_co2_ratio, vmax=max_co2_ratio))
sm.set_array([])  # You need to set a dummy array for the scalar mappable
cbar = plt.colorbar(sm, orientation='vertical', pad=0.02)
cbar.set_label('Equivalent Person CO2 Emissions per Capita in France')
plt.savefig('mix sales_by_country_co2info_plot.png')

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

# Take the mean of CO2 emission ratio for each product
mean_co2_ratio_by_product = aggregated_df.groupby('id_product')['CO2 Emission Ratio French'].mean()

# Plotting histogram
plt.figure(figsize=(12, 8))
sns.histplot(mean_co2_ratio_by_product, bins=30, kde=False, color='skyblue')
plt.title('Mean CO2 Emission Ratio Distribution by Product')
plt.xlabel('Equivalent Person CO2 Emissions per Capita in France')
plt.ylabel('Frequency')
plt.savefig('mix co2_emission_ratio_product_plot.png')
plt.show()


In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

# Aggregate data
aggregated_product_df = aggregated_df.groupby('id_product').agg({'Sales': 'sum', 'CO2 Emission Ratio French': 'mean'}).reset_index()

# Filter products with sales above 50
filtered_df = aggregated_product_df[aggregated_product_df['Sales'] > 50].copy()

# Take the mean of CO2 emission ratio for each product (after filtering)
mean_co2_ratio_by_product_filtered = filtered_df.groupby('id_product')['CO2 Emission Ratio French'].mean()

# Plotting histogram for filtered products
plt.figure(figsize=(12, 8))
sns.histplot(mean_co2_ratio_by_product_filtered, bins=30, kde=False, color='skyblue')
plt.title('Mean CO2 Emission Ratio Distribution by Product (Sales > 50)')
plt.xlabel('Mean CO2 Emission Ratio French')
plt.ylabel('Frequency')
plt.show()
